In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
data = pd.read_csv('./data/test_aus_data.csv').values.astype(floatX).T[np.newaxis][:,1:]/100

In [3]:
# data = data[:,1:] - data[:,:-1]

In [4]:
stds = (data[0,1:] - data[0,:-1]).std(axis=0)
print(stds)
data /= stds

[0.0186606  0.01934265 0.00918617]


In [5]:
data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
data.shape

(1, 36, 6)

In [6]:
np.any(np.isnan(data))

False

In [7]:
NUM_STEPS = data.shape[1]

with tf.variable_scope('walk_ord'):
    s1_prior_d = LogNormal(1, mu=math.log(0.01), sigma=6.)

    with tf.variable_scope('s1_inference', dtype=floatX):
        mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(s1_prior_d.mu))
        logsigma = tf.get_variable('logsigma', shape=[1], 
                                   initializer=tf.constant_initializer(min(math.log(s1_prior_d.sigma), -1.)))
        sigma = tf.exp(logsigma)
        s1_d = LogNormal(1, mu=mu, sigma=sigma)
    
    s1 = s1_d.sample()
    tf.summary.scalar('s1_ord', s1[0])
    tf.add_to_collection('logdensities', s1_d.logdens(s1))
    
    s1_prior = s1_prior_d.logdens(s1)
    tf.add_to_collection('priors', s1_prior)
    
s0 = 80.
dim = [3,3*2+1]

K = dim[0] * dim[1]

PWalk = NormalRW(dim=None, sigma0=s0, sigma=s1, name='OrdWalk')

In [8]:
#model coefficients
gvar = GVAR(dim=dim[0]*dim[1], len=NUM_STEPS, name='coef_rw_inference')
outputs = gvar.sample()

tf.add_to_collection('logdensities', gvar.logdens)

In [9]:
outputs

<tf.Tensor 'coef_rw_inference_1/VAR/strided_slice_36:0' shape=(1, 36, 21) dtype=float64>

In [10]:
prior_ord = PWalk.logdens(outputs, reduce=False)

Init_distr.mu is not None


In [11]:
obs_d_prior = LogNormal(dim=None, mu=math.log(0.5), sigma=0.5)

with tf.variable_scope('obs_d_inference', dtype=floatX):
    mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(math.log(0.5)))
    logsigma = tf.get_variable('logsigma', shape=[1], initializer=tf.constant_initializer(-5))
    sigma = tf.exp(logsigma)
    
    obs_d_post = LogNormal(1, mu=mu, sigma=sigma)
    obs_ds = obs_d_post.sample()
    
    obs_ds_logdens = obs_d_post.logdens(obs_ds)
    tf.add_to_collection('logdensities', obs_ds_logdens)
    tf.add_to_collection('priors', obs_d_prior.logdens(obs_ds))
    
    tf.summary.scalar('observ_sigma', obs_ds[0])

In [12]:
def predict(observable_mask):
    out = tf.reshape(outputs, [NUM_STEPS, dim[0], dim[1]])
    
    def step(prev, x):
        mask = x[0]
        prev_pred = tf.where(mask, x[1], prev)[tf.newaxis]
        params = x[2]
        
        d0 = params[:,:dim[0]]
        d1 = params[:,dim[0]:2*dim[0]]
        
        pp1 = prev_pred[:,:dim[0]]
        pp0 = prev_pred[:,dim[0]:2*dim[0]]
        
        new_pred = tf.matmul(pp0, d0)[0] + tf.matmul(pp1, d1)[0]+ params[:,-1] + pp1[0]
        new_pred = tf.concat([new_pred, pp1[0]], axis=0)
        return new_pred
    
    ar = tf.scan(step, [observable_mask, data[0], out], initializer=tf.zeros([2*dim[0]], dtype=floatX))
    return ar

In [13]:
data.shape[1]

36

In [14]:
stop_time = tf.placeholder_with_default(NUM_STEPS, ())
predict_mask = tf.range(0, data.shape[1], dtype=tf.int32) < stop_time
#используем в предсказаниях ровно stop_time последних наблюдений

In [15]:
preds = predict(predict_mask)

In [16]:
preds

<tf.Tensor 'scan/TensorArrayStack/TensorArrayGatherV3:0' shape=(36, 6) dtype=float64>

In [17]:
obs_d = Normal(dim=None, sigma=obs_ds, mu=0)

diffs = preds[:-1] - data[0,1:]
diffs = diffs[:,:dim[0]]

logl = obs_d.logdens(diffs, reduce=False)
logl *= tf.cast(predict_mask[:-1], floatX)[:,tf.newaxis]

logl = tf.reduce_sum(logl)
tf.add_to_collection('priors', logl)

In [18]:
diffs

<tf.Tensor 'strided_slice_1:0' shape=(35, 3) dtype=float64>

In [19]:
tf.summary.scalar('logl', logl)

<tf.Tensor 'logl:0' shape=() dtype=string>

In [20]:
def create_kl():
    logdens = tf.reduce_sum(tf.get_collection('logdensities'))
    prior = tf.reduce_sum(tf.get_collection('priors'))
    
    param_walk_prior = tf.reduce_sum(prior_ord)
    KLd = -prior + logdens - param_walk_prior
    KLd /= NUM_STEPS*dim[0]*dim[1]
    return KLd

In [21]:
tf.get_collection('logdensities')

[<tf.Tensor 'walk_ord/LogNormal_2/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'coef_rw_inference_1/VAR/logdens/add:0' shape=() dtype=float64>,
 <tf.Tensor 'obs_d_inference/LogNormal_2/Sum:0' shape=() dtype=float64>]

In [22]:
tf.get_collection('priors')

[<tf.Tensor 'walk_ord/LogNormal_3/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'obs_d_inference/LogNormal_3/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'Sum:0' shape=() dtype=float64>]

In [23]:
kl = create_kl()

In [24]:
kl

<tf.Tensor 'truediv:0' shape=() dtype=float64>

In [25]:
tf.summary.scalar('kl', kl)

<tf.Tensor 'kl:0' shape=() dtype=string>

In [26]:
opt = tf.train.AdamOptimizer(0.0001).minimize(kl)

In [27]:
summary = tf.summary.merge_all()

In [28]:
init = tf.global_variables_initializer()

In [29]:
sess = tf.InteractiveSession()

In [30]:
init.run()

In [32]:
writer = tf.summary.FileWriter('/tmp/hier/teeest')

In [35]:
for epoch in range(10000):
    for _ in range(100):
        sess.run(opt)
    s = sess.run(summary)
    writer.add_summary(s)

KeyboardInterrupt: 